In [2]:
import glob

def parse_data(data):
    result = []
    current_character = None
    current_line = ""

    for line in data.split("â–¶"):
        parts = line.split(":")
        character = parts[0].replace("â–¶", "").strip()
        dialogue = ":".join(parts[1:]).split("(")[0].strip()

        if character == current_character:
            current_line += " " + dialogue.replace('\n', ' ')
        else:
            if current_character is not None:
                result.append(f'"{current_character}": "{current_line.strip()}"')
            current_character = character
            current_line = dialogue.replace('\n', ' ')

    if current_character is not None:
        result.append(f'"{current_character}": "{current_line.strip()}"')

    return result

read_files = glob.glob("data/shows/Kono")

with open("result.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())

data = open("result.txt", "r").read()
parsed_data = parse_data(data)
print(parsed_data)

PermissionError: [Errno 13] Permission denied: 'data/shows/Kono'

In [2]:
import csv

output_data = []

for i in range(1, len(parsed_data)):
    if '"CHARNAME"' in parsed_data[i].split(":")[0]:
        prior_line = parsed_data[i-1].split('": "')[1].strip('"')
        megumin_line = parsed_data[i].split('": "')[1].strip('"')
        output_data.append([prior_line, megumin_line])

print(output_data)
with open('mega_output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_data)

[['Huh?', 'The mobile fortress, Destroyer... Fighting that thing is reckless, at best.'], ["We have that. If we need firepower, we have her, don't we? The crazy one. Thats's right. The crazy one! We do have the crazy girl! Stare...", "Hey, wait! If you're talking about me, don't call me by that nickname! Otherwise, I'll prove just how crazy I am right here and now!"], ['Your explosion magic is the greatest firepower this city has. What do you say, Megumin?', "I don't think even my explosion magic can take it down in a single blow."], ["That's not funny! That's not funny! Kazuma, are you okay over there?!", "I'm okay. I am strong. I am strong."], ["It's not looking good.", 'I am the finest of crimson demons, a genius magician...'], ['Hey, calm down. Try not to overthink it.', "I-I'll blow... blow it away with my explosion magic!"], ["Hey, Megumin! Hey! Is your love for explosion magic the real deal?! If you're outdone by Wiz, you'll be a disgrace! Is your explosion magic so wimpy, it ca

In [11]:
import csv
import json

def transform_csv(input_file, output_file):
    with open(input_file, 'r') as csv_file, open(output_file, 'w') as json_file:
        reader = csv.reader(csv_file)
        system_content = ""

        for row in reader:
            prompt, response = row
            data = {
                "messages": [
                    {"role": "system", "content": system_content},
                    {"role": "user", "content": prompt.replace('"', '')},
                    {"role": "assistant", "content": response.replace('"', '')}
                ]
            }
            json.dump(data, json_file)
            json_file.write('\n')

# Usage
input_file = 'mega_output.csv'
output_file = 'gpt_tune_output.jsonl'
transform_csv(input_file, output_file)